In [3]:
import numpy as np

from numpy.linalg import inv

from watermatrices import Amat, Bmat, yvec

In [348]:
size = Amat.shape[0]

E = np.vstack((np.hstack((Amat, Bmat)), np.hstack((Bmat, Amat))))

S = np.eye(size*2)
S[size:, size:] = -S[size:, size:]

z = np.hstack((yvec,-yvec))



In [342]:
# maxnorm is the maximum absolute row sum of the matrix.


maxnorm = lambda M: np.abs(M).sum(axis=1).max()

cond = lambda M: maxnorm(M) * maxnorm(inv(M))

print("Condition number of E: ", cond(E))

Condition number of E:  59.16590717093859


In [357]:
ωs = [0.800, 1.146, 1.400]


In [46]:
for ω in ωs:
    print("ω = ", ω)
    print("Condition number of E - ωS: ", cond(E-ω*S))

ω =  0.8
Condition number of E - ωS:  327.8167042420992
ω =  1.146
Condition number of E - ωS:  152679.26875233962
ω =  1.4
Condition number of E - ωS:  227.19443667104443


# LU Decomp

In [400]:
def same(a,b):
    return np.isclose(a,b).all()

def pivot(M):
    I = np.eye(M.shape[0])
    best = np.argmax(M[:,0])

    I[[0,best]] = I[[best,0]]

    return I


def decomp(M):
    assert M.shape[0] == M.shape[1], "Matrix should be square"
    I = lambda: np.eye(M.shape[0])

    U = M.copy()
    L = I()
    
    for i in range(M.shape[0]-1):
        # P = I()
        # P[i:,i:] = pivot(M[i:,i:])
        # M = M@P

        coeff = I()

        coeff[(i+1):,i] = -U[(i+1):,i]/U[i,i]

        L[(i+1):,i] = -coeff[(i+1):,i]
        U = coeff@U

    assert same(L@U, M), "Decomposition did not work"

    return U, L



M = np.array([2, 1, 1, 4, 1, 4, -6, -5, 3])
M = M.reshape((3,3))
z = [4,11,4]

print(M, end= "\n\n")
    
d = decomp(M)

print(*d,sep="\n")


[[ 2  1  1]
 [ 4  1  4]
 [-6 -5  3]]

[[ 2.  1.  1.]
 [ 0. -1.  2.]
 [ 0.  0.  2.]]
[[ 1.  0.  0.]
 [ 2.  1.  0.]
 [-3.  2.  1.]]


In [401]:
def forward_substitute(L, z):
    y = np.empty_like(z)#.astype(float)

    for k in range(L.shape[0]):
        y[k] = z[k] - (L[k, 0:k] * y[0:k]).sum()

    return y


def backward_substitute(U, y):
    x = np.empty_like(y)#.astype(float)
    N = U.shape[0]
    for k in range(N)[::-1]:
        x[k] = (y[k] - (U[k, (k+1):N] * x[(k+1):N]).sum()) / U[k,k]
    return x

fw = forward_substitute(d[1],z)


print(same(fw, np.linalg.solve(d[1],z)))

bw = backward_substitute(d[0],fw)

print(same(bw, np.linalg.solve(M,z)))


True
True


In [402]:
def solve_alpha(E, S, z, omega):
    M = E-omega*S
    L, U = decomp(M)
    forward = forward_substitute(L, z)
    
    x = backward_substitute(U, forward)
    alpha = (z*x).sum()
    return alpha

In [403]:
size = Amat.shape[0]

E = np.vstack((np.hstack((Amat, Bmat)), np.hstack((Bmat, Amat))))

S = np.eye(size*2)
S[size:, size:] = -S[size:, size:]

z = np.hstack((yvec,-yvec))


ωs = [0.800, 1.146, 1.400]
dω = 0.5e-3

for ω in ωs:
    for d in [-dω, dω]:
        print(ω, d)
        print(solve_alpha(E, S, z, ω + d))




0.8 -0.0005
2.6490514214431427
0.8 0.0005
2.6490514214431427
1.146 -0.0005
2.6490514214431427
1.146 0.0005
2.6490514214431427
1.4 -0.0005
2.6490514214431427
1.4 0.0005
2.6490514214431427
